# **1. Install and Import Dependencies**

In [1]:
!pip3 install transformers
!pip3 install datasets
!pip3 install evaluate

In [18]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


# **2. Load dataset**

In [2]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import pandas as pd
import numpy as np
import os

df = pd.read_csv('IMDB Dataset.csv')
df['sentiment'] = np.where(df['sentiment'] == "positive", 1, df['sentiment'])
df['sentiment'] = np.where(df['sentiment'] == "negative", 0, df['sentiment'])

df_train = df[:int(len(df)*0.8)]
df_test  = df[int(len(df)*0.8):]

df_train = df_train.sample(4000, random_state=1)
df_test  = df_test.sample(1000, random_state=1)

raw_train_dataset = Dataset.from_pandas(df_train)
raw_test_dataset = Dataset.from_pandas(df_test)
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
  return tokenizer(example['review'], truncation=True)

tokenized_train_dataset = raw_train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset =  raw_test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [3]:
tokenized_train_dataset = tokenized_train_dataset.rename_column("sentiment", "labels")
tokenized_test_dataset = tokenized_test_dataset.rename_column("sentiment", "labels")
print(tokenized_train_dataset)
print(tokenized_test_dataset)


Dataset({
    features: ['review', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4000
})
Dataset({
    features: ['review', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})


In [4]:
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["review", "__index_level_0__"])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["review", "__index_level_0__"])
print(tokenized_train_dataset)
print(tokenized_test_dataset)

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4000
})
Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})


In [5]:
tokenized_train_dataset.set_format("torch")
tokenized_test_dataset.set_format("torch")

print(tokenized_train_dataset)
data_collator = DataCollatorWithPadding(tokenizer)

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4000
})


In [6]:
from torch.utils.data import DataLoader

In [7]:
train_dataloader = DataLoader(
    tokenized_train_dataset, shuffle=True, batch_size=8, collate_fn=data_collator, 
)

test_dataloader = DataLoader(
    tokenized_test_dataset, batch_size = 8, collate_fn=data_collator
)

In [8]:
for batch in train_dataloader:
  break
print({k: v.shape for k,v in batch.items()})

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]), 'input_ids': torch.Size([8, 386]), 'token_type_ids': torch.Size([8, 386]), 'attention_mask': torch.Size([8, 386])}


# **2. Instantiate Model**

In [9]:
from transformers import AutoModelForSequenceClassification
checkpoint = "bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model.to(device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [11]:
batch = batch.to(device)
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.6375, device='cuda:0', grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [12]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

C:\Users\Alex\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
loss = outputs.loss
loss.backward()
optimizer.step()

optimizer.zero_grad()

In [14]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [15]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
  for batch in train_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

  0%|          | 0/1500 [00:00<?, ?it/s]


# **3. Encode and Calculate Sentiment**

In [23]:
tokens = tokenizer.encode("I hated this movie, it was worse than the second film.", return_tensors='pt')

In [24]:
result = model(tokens.to(device))

In [25]:
result.logits

tensor([[ 2.9293, -2.3870]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [26]:
int(torch.argmax(result.logits))

0

# **5. Evaluation**

In [41]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

import evaluate

accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")


model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    accuracy_metric.add_batch(predictions=predictions, references=batch["labels"])
    precision_metric.add_batch(predictions=predictions, references=batch["labels"])
    recall_metric.add_batch(predictions=predictions, references=batch["labels"])

print(accuracy_metric.compute())
print(precision_metric.compute())
print(recall_metric.compute())

{'accuracy': 0.904}
{'precision': 0.904572564612326}
{'recall': 0.904572564612326}


In [9]:
import joblib

# save the model to disk
filename = 'finetuned_model.sav'

#model, tokenizer = joblib.load('./finetuned_model.sav')
model.to("cpu")

joblib.dump(value=[model, tokenizer], filename="./finetuned_model.sav", compress=0)



['./finetuned_model.sav']